In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(legacy="1.25")

### Загрузка данных

In [2]:
df = pd.read_csv('data/cities.csv')

In [3]:
# Координаты административных центров
centers = {
    'Москва': (55.7558, 37.6173),
    'Новосибирск': (55.0084, 82.9357),
    'Владивосток': (43.1155, 131.8855)
}

In [5]:
# Функция для расчета расстояния между двумя точками на Земле
def haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6371  # Радиус Земли в километрах
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_km = RADIUS * c
    return total_km

# Функция для безопасного преобразования
def safe_convert_population(population):
    try:
        return int(population.replace(' ', ''))
    except ValueError:
        return np.nan

In [6]:
df['Население'] = df['Население'].apply(safe_convert_population)

In [8]:
# 1. Города в Центральном федеральном округе в радиусе 100 км от Москвы с населением <= 50т.
central_federal_district = df.query("`Федеральный округ` == 'Центральный' and `Население` <= 50000")
central_federal_district['Расстояние'] = haversine(
    55.7558, 37.6173, 
    central_federal_district['Широта'].values, 
    central_federal_district['Долгота'].values
)
result_central = central_federal_district[central_federal_district['Расстояние'] <= 100]
result_central.index.size

33

In [9]:
# 2. Города в Сибирском федеральном округе в радиусе от 150 до 250 км от Новосибирска с населением от 100т. до 200т.
siberian_federal_district = df.query("`Федеральный округ` == 'Сибирский' and 100_000 <= Население <= 200_000")
siberian_federal_district['Расстояние'] = haversine(
        55.0084, 82.9357, 
        siberian_federal_district['Широта'].values, 
        siberian_federal_district['Долгота'].values
)
result_siberian = siberian_federal_district[
    (siberian_federal_district['Расстояние'] >= 150) & 
    (siberian_federal_district['Расстояние'] <= 250)
]
result_siberian

C:\Users\Admin\AppData\Local\Temp\ipykernel_12036\1093539059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siberian_federal_district['Расстояние'] = haversine(


,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население,Расстояние
317,652500.0,обл,Кемеровская,NaN,NaN,г,Ленинск-Кузнецкий,NaN,NaN,4200001000000,...,0,32419000000,3.271900e+10,4212,UTC+7,54.667409,86.179672,Сибирский,101666.0,211.143393
961,636000.0,обл,Томская,NaN,NaN,г,Северск,NaN,NaN,7000000300000,...,0,69541000000,6.974100e+10,7024,UTC+7,56.603114,84.880969,Сибирский,108466.0,214.972713


In [10]:
# 3. Города в Дальневосточном федеральном округе ближе всего к Владивостоку с населением от 10т. до 20т.
dalnevostochniy_federal_district = df.query("`Федеральный округ` == 'Дальневосточный' and 10000 <= Население <= 20000")
dalnevostochniy_federal_district['Расстояние'] = haversine(
    43.1155, 131.8855, 
    dalnevostochniy_federal_district['Широта'].values, 
    dalnevostochniy_federal_district['Долгота'].values
)
result_dalnevostochniy = dalnevostochniy_federal_district.nsmallest(5, 'Расстояние')
result_dalnevostochniy

C:\Users\Admin\AppData\Local\Temp\ipykernel_12036\3048833235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dalnevostochniy_federal_district['Расстояние'] = haversine(


,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население,Расстояние
1006,682970.0,край,Хабаровский,NaN,NaN,г,Бикин,NaN,NaN,2700000400000,...,0,8406000000,8.609101e+06,2720,UTC+10,46.818624,134.255105,Дальневосточный,17156.0,451.946347
1007,682950.0,край,Хабаровский,р-н,Вяземский,г,Вяземский,NaN,NaN,2700700100000,...,1,8217501000,8.617101e+06,2720,UTC+10,47.535442,134.755370,Дальневосточный,14556.0,540.166223
17,676870.0,обл,Амурская,р-н,Завитинский,г,Завитинск,NaN,NaN,2800500100000,...,1,10221501000,1.062110e+07,2813,UTC+9,50.106468,129.439290,Дальневосточный,11481.0,799.381979
807,694740.0,обл,Сахалинская,р-н,Невельский,г,Невельск,NaN,NaN,6500700100000,...,1,64420000000,6.472800e+10,6509,UTC+11,46.652731,141.863174,Дальневосточный,11682.0,878.118083
803,694051.0,обл,Сахалинская,р-н,Долинский,г,Долинск,NaN,NaN,6500300100000,...,1,64212000000,6.471200e+07,6504,UTC+11,47.325590,142.794599,Дальневосточный,12200.0,973.025470


In [11]:
# 4. Три самых многочисленных округа с радиусом 100 км от административного центра, 
# не считая ЦФО.

# Фильтруем данные, исключая ЦФО
df_non_cfo = df[~df['Федеральный округ'].str.contains('Центральный')]

In [13]:
# Находим административные центры (где Признак центра района или региона = 1)
admin_centers = df_non_cfo[df_non_cfo['Признак центра района или региона'] == 1]
admin_centers

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
7,658420.0,край,Алтайский,р-н,Локтевский,г,Горняк,NaN,NaN,2202700100000,...,4: город,1,1225501000,1.625101e+06,2209,UTC+7,50.997903,81.464306,Сибирский,13040.0
9,658480.0,край,Алтайский,р-н,Змеиногорский,г,Змеиногорск,NaN,NaN,2201500100000,...,4: город,1,1214501000,1.614101e+06,2209,UTC+7,51.158023,82.187248,Сибирский,10569.0
10,658700.0,край,Алтайский,р-н,Каменский,г,Камень-на-Оби,NaN,NaN,2201800100000,...,4: город,1,1216501000,1.616101e+09,2207,UTC+7,53.791545,81.354517,Сибирский,41787.0
17,676870.0,обл,Амурская,р-н,Завитинский,г,Завитинск,NaN,NaN,2800500100000,...,4: город,1,10221501000,1.062110e+07,2813,UTC+9,50.106468,129.439290,Дальневосточный,11481.0
21,676011.0,обл,Амурская,р-н,Сковородинский,г,Сковородино,NaN,NaN,2801700100000,...,4: город,1,10249501000,1.064910e+07,2808,UTC+9,53.987100,123.943729,Дальневосточный,9561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,678450.0,Респ,Саха /Якутия/,у,Нюрбинский,г,Нюрба,NaN,NaN,1402100100000,...,4: город,1,98409000000,9.862610e+07,1445,UTC+9,63.282877,118.324325,Дальневосточный,10156.0
1087,678100.0,Респ,Саха /Якутия/,у,Олекминский,г,Олекминск,NaN,NaN,1402300100000,...,4: город,1,98241501000,9.864110e+07,1450,UTC+9,60.375820,120.406013,Дальневосточный,9487.0
1088,678000.0,Респ,Саха /Якутия/,у,Хангаласский,г,Покровск,NaN,NaN,1403200100000,...,4: город,1,98420000000,9.864410e+07,1448,UTC+9,61.484354,129.148220,Дальневосточный,9495.0
1089,678790.0,Респ,Саха /Якутия/,у,Среднеколымский,г,Среднеколымск,NaN,NaN,1402500100000,...,4: город,1,98246501000,9.864610e+07,1448,UTC+11,67.458166,153.706951,Дальневосточный,3525.0


In [14]:
# Создаем список для хранения результатов
results = []

# Для каждого административного центра вычисляем расстояние до всех других регионов
for index, center in admin_centers.iterrows():
    center_lat = center['Широта']
    center_lon = center['Долгота']

 # Вычисляем расстояние до всех регионов
    df_non_cfo['Расстояние'] = haversine(center_lat, center_lon, df_non_cfo['Широта'], df_non_cfo['Долгота'])
    # Фильтруем регионы в радиусе 100 км
    nearby_regions = df_non_cfo[df_non_cfo['Расстояние'] <= 100]
    # Считаем население для этих регионов
    total_population = nearby_regions['Население'].sum()
    results.append({'Федеральный округ': center['Федеральный округ'], 'Население': total_population})

C:\Users\Admin\AppData\Local\Temp\ipykernel_12036\3584867297.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_cfo['Расстояние'] = haversine(center_lat, center_lon, df_non_cfo['Широта'], df_non_cfo['Долгота'])


In [16]:
# Преобразуем результаты в DataFrame
results_df = pd.DataFrame(results)
results_df.head()
# Находим три самых многочисленных округа
top_counties = results_df.groupby('Федеральный округ')['Население'].sum().nlargest(3)
top_counties

,Федеральный округ,Население
0,Сибирский,169995.0
1,Сибирский,169995.0
2,Сибирский,41787.0
3,Дальневосточный,31980.0
4,Дальневосточный,9561.0


Федеральный округ
Приволжский        93491451.0
Северо-Западный    44687819.0
Южный              35827002.0
Name: Население, dtype: float64